<a href="https://colab.research.google.com/github/yyoonyoon/201935291-/blob/main/4%EC%9E%A5_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 준비. 처음 데이터 5줄 출력.

In [38]:
import pandas as pd

flower = pd.read_csv('https://raw.githubusercontent.com/jmnote/zdata/master/R/iris.csv')
flower.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


클래스 추출. 데이터에 어떤 클래스가 있는지 확인.

In [39]:
print(pd.unique(flower['Species']))

['setosa' 'versicolor' 'virginica']


In [40]:
flower_input = flower[["Sepal.Length","Sepal.Width","Petal.Length","Petal.Width"]].to_numpy()

In [41]:
flower_target = flower["Species"].to_numpy()

In [42]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    flower_input, flower_target, stratify=flower_target, random_state=42)

In [43]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [44]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))  #분류문제에서 score 메서드가 반환하는 값은 정확도이고 회귀 문제에서는 결정계수를 반환함.

0.9642857142857143
0.9210526315789473


In [45]:
print(kn.classes_)

['setosa' 'versicolor' 'virginica']


In [46]:
print(kn.predict(test_scaled[:5])) #5개에 대한 확률 예측

['setosa' 'versicolor' 'versicolor' 'versicolor' 'setosa']


In [47]:
import numpy as np

proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

distances, indexes = kn.kneighbors(test_scaled[3:4]) #그냥 [3]으로 써도 됨.
print(train_target[indexes])


[[1.     0.     0.    ]
 [0.     0.6667 0.3333]
 [0.     1.     0.    ]
 [0.     1.     0.    ]
 [1.     0.     0.    ]]
[['versicolor' 'versicolor' 'versicolor']]


In [48]:
#로지스틱 회귀로 다중 분류 시행
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9821428571428571
0.9473684210526315


In [49]:
print(lr.predict(test_scaled[:5]))

['setosa' 'versicolor' 'versicolor' 'versicolor' 'setosa']


In [50]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.999 0.001 0.   ]
 [0.003 0.938 0.059]
 [0.004 0.994 0.001]
 [0.    0.995 0.004]
 [0.998 0.002 0.   ]]


In [51]:
print(lr.classes_)
print(lr.coef_.shape, lr.intercept_.shape)

['setosa' 'versicolor' 'virginica']
(3, 4) (3,)


In [52]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ 14.49   7.22 -21.71]
 [ -2.88   2.82   0.06]
 [ -1.46   3.99  -2.52]
 [ -3.4    4.44  -1.04]
 [ 13.45   7.44 -20.9 ]]


In [53]:
from scipy.special import softmax #axis=1 은 행, 0은 열. 지정하지 않으면 모든 값에 대해 소프트맥스 계산.

proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[0.999 0.001 0.   ]
 [0.003 0.938 0.059]
 [0.004 0.994 0.001]
 [0.    0.995 0.004]
 [0.998 0.002 0.   ]]


In [54]:
from sklearn.linear_model import SGDClassifier

In [55]:
sc = SGDClassifier(loss='log_loss', max_iter=10, random_state=42)
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

0.8392857142857143
0.7631578947368421


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [56]:
sc.partial_fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target))
print(sc.score(test_scaled, test_target))

0.9285714285714286
0.8421052631578947
